In [6]:
import glob
import os, re, numpy as np, tensorflow as tf


txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


In [7]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,¿])",r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

print(preprocess_sentence("This @_is ;;;sample               sentence."))

corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue

    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

len(corpus)

def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,     # 12000 단어를 기억
        filters=' ',        # 이미 문장을 정제했기 때문에 사용안함.
        oov_token="<unk>"   # 12000단어에 포함되지 못하면 <unk>로변환.
    )

    tokenizer.fit_on_texts(corpus)  # 내부의 단어장 완성
    tensor = tokenizer.texts_to_sequences(corpus)   # 텐서로 변환
    under = []
    for i in range(len(tensor)):
        if len(tensor[i]) <= 16:
            under.append(tensor[i])
    tensor = tf.keras.preprocessing.sequence.pad_sequences(under, padding='post')
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

print(src_input[0])
print(tgt_input[0])

from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input,                                                          
                                                          test_size=0.2,
                                                          random_state=42)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

<start> this is sample sentence . <end>
[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...    3    0    0]
 ...
 [   2    5  107 ...    0    0    0]
 [   2  261  200 ...    3    0    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7ff24c4b11f0>
[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0
    0]
[  50    5   91  297   65   57    9  969 6042    3    0    0    0    0
    0]
Source Train: (128205, 15)
Target Train: (128205, 15)


In [8]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 15), (256, 15)), types: (tf.int32, tf.int32)>

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)

        return out

embedding_size = 256
hidden_size = 1024
lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

In [14]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
lyricist(src_sample)

<tf.Tensor: shape=(256, 15, 12001), dtype=float32, numpy=
array([[[ -4.303407  ,   2.8869326 , -11.684994  , ..., -10.487542  ,
         -10.768728  , -11.649715  ],
        [ -4.8558927 ,   2.0634398 , -13.817428  , ..., -12.42427   ,
         -11.137197  , -13.657329  ],
        [ -5.376406  ,   1.9988382 , -14.687974  , ..., -13.028472  ,
         -12.711949  , -14.561758  ],
        ...,
        [  9.831456  ,  -5.1378813 , -19.527908  , ..., -27.280989  ,
         -41.656193  , -19.31424   ],
        [  9.828184  ,  -5.1402173 , -19.537666  , ..., -27.300316  ,
         -41.665398  , -19.323847  ],
        [  9.824823  ,  -5.141843  , -19.542265  , ..., -27.301754  ,
         -41.66736   , -19.32785   ]],

       [[ -4.303407  ,   2.8869326 , -11.684994  , ..., -10.487542  ,
         -10.768728  , -11.649715  ],
        [ -4.3089337 ,   1.4012109 , -14.388656  , ..., -14.429604  ,
         -11.546996  , -14.213851  ],
        [ -6.1410847 ,   1.3486375 , -16.6927    , ..., -13.798

In [15]:
lyricist.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_3 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_1 (Dense)              multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [11]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

lyricist.compile(loss=loss, optimizer=optimizer)
lyricist.fit(dataset, epochs=10)

Epoch 1/10
500/500 [==============================] - 96s 179ms/step - loss: 3.3213
Epoch 2/10
500/500 [==============================] - 91s 181ms/step - loss: 2.8815
Epoch 3/10
500/500 [==============================] - 91s 182ms/step - loss: 2.7309
Epoch 4/10
500/500 [==============================] - 91s 181ms/step - loss: 2.6120
Epoch 5/10
500/500 [==============================] - 90s 181ms/step - loss: 2.5127
Epoch 6/10
500/500 [==============================] - 90s 180ms/step - loss: 2.4250
Epoch 7/10
500/500 [==============================] - 91s 181ms/step - loss: 2.3443
Epoch 8/10
500/500 [==============================] - 91s 181ms/step - loss: 2.2693
Epoch 9/10
500/500 [==============================] - 91s 182ms/step - loss: 2.1980
Epoch 10/10
500/500 [==============================] - 91s 182ms/step - loss: 2.1311


In [24]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20): 
    test_input = tokenizer.texts_to_sequences([init_sentence]) 
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64) 
    end_token = tokenizer.word_index["<end>"] 
    
    while True: 
        predict = model(test_tensor) 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1) 
        if predict_word.numpy()[0] == end_token: break 
        if test_tensor.shape[1] >= max_len: break 
    
    generated = "" 
            
    for word_index in test_tensor[0].numpy(): 
        generated += tokenizer.index_word[word_index] + " " 
    return generated

In [25]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m a motherfucking monster <end> '